In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

In [ ]:
import importlib

In [ ]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:10<01:39,  1.82it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:47<00:39,  2.50it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [01:05<01:18,  1.13s/it]

...

 79%|███████▉  | 158/200 [01:22<00:22,  1.88it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [02:08<00:00,  1.55it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 2664.85it/s]


In [ ]:
trading_data.packed_val_x[0]

PackedSequence(data=tensor([[ 0.0000e+00,  2.5554e+06, -1.0000e+00,  ...,  1.0000e+00,
          2.4081e+03,  1.0000e+00],
        [ 0.0000e+00,  2.7470e+05,  1.0000e+00,  ...,  1.0000e+00,
          1.7408e+02,  1.0000e+00],
        [ 0.0000e+00,  4.1553e+05,  1.0000e+00,  ...,  1.0003e+00,
          2.2485e+04,  1.0000e+00],
        ...,
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  9.9962e-01,
          1.0968e+02,  9.9962e-01],
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  9.9673e-01,
          2.1029e+05,  9.9664e-01],
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  1.0019e+00,
          8.6312e+04,  1.0017e+00]], device='cuda:0'), batch_sizes=tensor([200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]), sorted_in

In [ ]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNet(12,hidden_size).to('cuda:0')
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [ ]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [13]:
config_static = {'learning_rate':0.00002, 'hidden_size':64}

In [14]:
model = model_pipeline(trading_data, config_static)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


  0%|          | 0/10000 [00:00<?, ?it/s]

created path


Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_14604\2921910823.py", line 11, in model_pipeline
    training_testing.train_model(trading_df,model,config,optimizer,criterion)
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 46, in train_model
    loss.backward()
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\autograd\__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_l1_loss,▃▃▂▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▄▃▄▃▃▄▄▄▄▄▄▅▆█▇▇▇▆▇▇
epoch_loss,▃▃▂▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▄▃▄▃▃▄▄▄▄▄▄▅▆█▇▇▇▆▇▇
loss_1,█▃▃▃▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▂▂▃▂▂▂▂▂▂
val_epoch_loss,▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▂▂▂▂▂▃▃▃▃▃▄▅█▆▇▆▅▆▇
val_epoch_loss_l1,▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▂▂▂▂▂▃▃▃▃▃▄▅█▆▇▆▅▆▇
epoch,621
epoch_loss,6.34466
loss_1,7.82644
val_epoch_loss,5.887


KeyboardInterrupt: 